In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.storagelevel import StorageLevel
import jieba
from operator import add

In [2]:
conf = SparkConf().setAppName("test").setMaster("local[*]")
sc = SparkContext(conf=conf)

In [3]:
content = "小明碩士畢業於建國中學後在台灣大學就讀"
result = jieba.cut(content,True)
print(list(result))
print(type(result))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.362 seconds.
Prefix dict has been built successfully.


['小', '明', '碩', '士', '畢', '業', '於', '建', '國', '中', '學', '後', '在', '台', '灣', '大', '學', '就', '讀']
<class 'generator'>


In [4]:
# read file 
file_rdd = sc.textFile("SogouQ.txt")

#split file
split_rdd = file_rdd.map(lambda x: x.split("\t"))

# cus split_rdd need to use more than one time, so we cache it.
split_rdd.persist(StorageLevel.DISK_ONLY)

# analysis user search 
# print(split_rdd.takeSample(True,3))
def context_jieba(data):
    # 使用jieba分詞
    seg = jieba.cut_for_search(data)
    l = list()
    for word in seg:
        l.append(word)
    return l

# 取出搜索詞並括號取代掉
context_rdd = split_rdd.map(lambda x: x[2].replace('[','').replace(']','').replace('+','').replace('.',''))

# 使用flatMap是為了解決分詞後會分很多組的問題
words_rdd = context_rdd.flatMap(context_jieba)

#print(words_rdd.collect())

#用map變成數組用以統計
final_words_rdd = words_rdd.map(lambda x:(x, 1))

#取出前N名,照理說要先進行資料清理把贅字符號移除
print(final_words_rdd.reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[1], ascending=False, numPartitions=1).take(5))

[('地震', 615), ('的', 492), ('汶川', 433), ('原因', 360), ('救灾', 338)]


In [5]:
#用戶關鍵詞組合分析
user_content_rdd = split_rdd.map(lambda x:(x[1],x[2]))
#對user搜尋進行分詞再根據ID組合
def extract_user_and_word(data):
    user_id = data[0]
    content = data[1].replace('[','').replace(']','').replace('+','').replace('.','')
    words = context_jieba(content)
    return_list = list()
    for word in words:
        return_list.append(user_id + "_" + word)
    return return_list

user_word_with_one_rdd = user_content_rdd.flatMap(extract_user_and_word)


#分組聚合排序
result = user_word_with_one_rdd.map(lambda x: (x,1)).\
    reduceByKey(lambda a, b: a + b).\
    sortBy(lambda x: x[1],ascending=False,numPartitions=1).\
    take(5)
print(result)

[('1011517038707826_主题', 27), ('7230120314300312_阿宾', 23), ('7230120314300312_全集', 21), ('7230120314300312_阅读', 20), ('9026201537815861_scat', 19)]


In [6]:
#熱門搜索時間段分析
time_rdd = split_rdd.map(lambda x: x[0])

#保留分精度
minute_with_one_rdd = time_rdd.map(lambda x: (x.split(":")[1],1))

#分組 聚合
minute_result = minute_with_one_rdd.reduceByKey(add).\
    sortBy(lambda x: x[1],ascending=False,numPartitions=1).\
    collect()
print(minute_result)

[('02', 1088), ('04', 1056), ('03', 1051), ('00', 1046), ('01', 1046), ('06', 1036), ('05', 1024), ('08', 1024), ('07', 999), ('09', 630)]


In [7]:


stu_info_list=[(1,"Amy",11),(2,"Bob",13),(3,"Chris",11),(4,"David",11)]
score_info_rdd = sc.parallelize([(1,"國語",90),(2,"數學",91),(3,"英語",92),(2,"英語",96),(2,"國語",91),(1,"數學",94),(3,"數學",95),(1,"英語",91),(3,"國語",91)])

# 未使用廣播變量

def map_func(data):
    id = data[0]
    name = ""
    for stu_info in stu_info_list:
        stu_id = stu_info[0]
        if id == stu_id:
            name = stu_info[1]
    return (name,data[1],data[2])


print(score_info_rdd.map(map_func).collect())

# 使用廣播變量

broadcast = sc.broadcast(stu_info_list)

def map_func(data):
    id = data[0]
    name = ""
    for stu_info in broadcast.value:
        stu_id = stu_info[0]
        if id == stu_id:
            name = stu_info[1]
    return (name,data[1],data[2])

print(score_info_rdd.map(map_func).collect())

[('Amy', '國語', 90), ('Bob', '數學', 91), ('Chris', '英語', 92), ('Bob', '英語', 96), ('Bob', '國語', 91), ('Amy', '數學', 94), ('Chris', '數學', 95), ('Amy', '英語', 91), ('Chris', '國語', 91)]
[('Amy', '國語', 90), ('Bob', '數學', 91), ('Chris', '英語', 92), ('Bob', '英語', 96), ('Bob', '國語', 91), ('Amy', '數學', 94), ('Chris', '數學', 95), ('Amy', '英語', 91), ('Chris', '國語', 91)]


In [9]:
rdd = sc.parallelize([1,2,3,4,5,6,7,8,9,10], 2)

count = 0

def map_func(data):
    global count
    count += 1
    print(count)
    
rdd.map(map_func).collect()
print(count)

0


In [10]:
rdd = sc.parallelize([1,2,3,4,5,6,7,8,9,10], 2)

count = sc.accumulator(0)

def map_func(data):
    global count
    count += 1
    print(count)
    
rdd.map(map_func).collect()
print(count)

10
